In [1]:
import os
import torch

from training.inference import get_latents
from datasets import tqdm

def generate_evaluation_dataset(model, dataloader, name, chunking=True, averaging=False, chunk_size=256):
    model.mask_ratio = 0.0
    latents, labels = get_latents(dataloader, model, chunking=chunking, averaging=averaging, chunk_size=chunk_size)
    print("Saving...")

    index = 0
    for latent, label in zip(latents, labels):
        directory = f"D:\\SongsDataset\\GTZAN\\latent_datasets\\{name}\\"
        os.makedirs(os.path.dirname(directory), exist_ok=True)

        if not averaging and chunking:
            for x in range(label.shape[0]):
                label_name = directory + f"{index}_label"
                latent_name = directory + f"{index}_data"
                torch.save(latent, latent_name + ".pt")
                torch.save(label, label_name + ".pt")
                index += 1
        else:
            label_name = directory + f"{index}_label"
            latent_name = directory + f"{index}_data"
            torch.save(latent, latent_name + ".pt")
            torch.save(label, label_name + ".pt")
            index += 1


In [2]:
from data.data_utils import GTZAN
from torch.utils.data import DataLoader

directory = "D:\\SongsDataset\\GTZAN\\genres_original\\"
GTZAN_dataset = GTZAN(directory)
GTZAN_dataloader = DataLoader(
    GTZAN_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=2,
    prefetch_factor=1
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [3]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-Sinusoidal\\Epoch-79.pt", weights_only=False)
generate_evaluation_dataset(model, GTZAN_dataloader, "Sinusoidal-Chunking-256", chunking=True, chunk_size=256, averaging=True)

100%|██████████| 999/999 [00:30<00:00, 33.09it/s]


Saving...


In [4]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-ALIBI\\Epoch-103.pt", weights_only=False)
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-256", chunking=True, chunk_size=256, averaging=True)

100%|██████████| 999/999 [00:29<00:00, 34.21it/s]


Saving...


In [27]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-512", chunking=True, chunk_size=512, averaging=True)

100%|██████████| 999/999 [00:35<00:00, 28.14it/s]


Saving...


In [28]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-1024", chunking=True, chunk_size=1024, averaging=True)

100%|██████████| 999/999 [00:29<00:00, 34.35it/s]


Saving...


In [29]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-2048", chunking=True, chunk_size=2048, averaging=True)

100%|██████████| 999/999 [00:38<00:00, 26.22it/s]


Saving...


In [17]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-4096", chunking=True, chunk_size=4096)

  0%|          | 0/999 [00:15<?, ?it/s]


RuntimeError: max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.

In [15]:
generate_evaluation_dataset(model, GTZAN_dataloader, "ALIBI-Chunking-No-Chunking", chunking=True, chunk_size=16384)

  0%|          | 0/999 [00:16<?, ?it/s]


RuntimeError: max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.

In [24]:
from data.data_utils import LatentDataset
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import tqdm

# ----- Training Loop -----
def train_model(model, dataloader, config, device="cuda"):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    model.train()
    for epoch in tqdm(range(config.num_epochs)):
        total_loss = 0
        for labels, data in dataloader:
            # if dataset returns one-hot, convert back to integer for CrossEntropy
            if labels.ndim > 1:
                labels = labels.argmax(dim=1)

            data = data.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        if epoch % 4 == 0:
            print(f"Epoch {epoch+1}/{config.num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

    return model

In [17]:
import torch.nn

# ----- Model -----
class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.linear(x)

In [34]:
from utils.Config import Config
from torch.utils.data import DataLoader

def test_latent_dataaset(dataset):
    directory = f"D:\\SongsDataset\\GTZAN\\latent_datasets\\{dataset}\\"

    num_classes = 10
    latent_dataset = LatentDataset(directory, num_classes=num_classes)
    latent_dataloader = DataLoader(
        latent_dataset,
        batch_size=64,
        shuffle=True,
    )

    model_name = f"LinearClassifier-{dataset}"
    config = Config(
            save_path=f"trained_models\\{model_name}\\",
            num_epochs=1024,
            learning_rate=1e-3,
            weight_decay=1e-3,
            num_workers=2,
            batch_size= 64,
            eval_batch_size=64,
            dtype=torch.float32
        )

    device = "cuda"
    model = LinearClassifier(128, num_classes).to(device)
    train_model(model, latent_dataloader, config, device=device)

In [35]:
dataset = "Sinusoidal-Chunking-256"
test_latent_dataaset(dataset)

  0%|          | 0/1024 [00:00<?, ?it/s]

Epoch 1/1024, Loss: 2.1795
Epoch 5/1024, Loss: 1.2623
Epoch 9/1024, Loss: 1.0303
Epoch 13/1024, Loss: 0.9230
Epoch 17/1024, Loss: 0.8548
Epoch 21/1024, Loss: 0.8119
Epoch 25/1024, Loss: 0.7707
Epoch 29/1024, Loss: 0.7443
Epoch 33/1024, Loss: 0.7243
Epoch 37/1024, Loss: 0.7032
Epoch 41/1024, Loss: 0.6867
Epoch 45/1024, Loss: 0.6779
Epoch 49/1024, Loss: 0.6594
Epoch 53/1024, Loss: 0.6557
Epoch 57/1024, Loss: 0.6393
Epoch 61/1024, Loss: 0.6281
Epoch 65/1024, Loss: 0.6190
Epoch 69/1024, Loss: 0.6167
Epoch 73/1024, Loss: 0.6026
Epoch 77/1024, Loss: 0.5960
Epoch 81/1024, Loss: 0.5854
Epoch 85/1024, Loss: 0.5906
Epoch 89/1024, Loss: 0.5787
Epoch 93/1024, Loss: 0.5769
Epoch 97/1024, Loss: 0.5724
Epoch 101/1024, Loss: 0.5676
Epoch 105/1024, Loss: 0.5646
Epoch 109/1024, Loss: 0.5569
Epoch 113/1024, Loss: 0.5494
Epoch 117/1024, Loss: 0.5506
Epoch 121/1024, Loss: 0.5449
Epoch 125/1024, Loss: 0.5418
Epoch 129/1024, Loss: 0.5424
Epoch 133/1024, Loss: 0.5325
Epoch 137/1024, Loss: 0.5346
Epoch 141/102

In [36]:
dataset = "ALIBI-Chunking-256"
test_latent_dataaset(dataset)

  0%|          | 0/1024 [00:00<?, ?it/s]

Epoch 1/1024, Loss: 2.2860
Epoch 5/1024, Loss: 1.2829
Epoch 9/1024, Loss: 1.0390
Epoch 13/1024, Loss: 0.9261
Epoch 17/1024, Loss: 0.8586
Epoch 21/1024, Loss: 0.8149
Epoch 25/1024, Loss: 0.7776
Epoch 29/1024, Loss: 0.7532
Epoch 33/1024, Loss: 0.7280
Epoch 37/1024, Loss: 0.7100
Epoch 41/1024, Loss: 0.6954
Epoch 45/1024, Loss: 0.6760
Epoch 49/1024, Loss: 0.6613
Epoch 53/1024, Loss: 0.6526
Epoch 57/1024, Loss: 0.6410
Epoch 61/1024, Loss: 0.6360
Epoch 65/1024, Loss: 0.6271
Epoch 69/1024, Loss: 0.6215
Epoch 73/1024, Loss: 0.6145
Epoch 77/1024, Loss: 0.6046
Epoch 81/1024, Loss: 0.5948
Epoch 85/1024, Loss: 0.5984
Epoch 89/1024, Loss: 0.5848
Epoch 93/1024, Loss: 0.5782
Epoch 97/1024, Loss: 0.5722
Epoch 101/1024, Loss: 0.5665
Epoch 105/1024, Loss: 0.5638
Epoch 109/1024, Loss: 0.5672
Epoch 113/1024, Loss: 0.5578
Epoch 117/1024, Loss: 0.5498
Epoch 121/1024, Loss: 0.5486
Epoch 125/1024, Loss: 0.5394
Epoch 129/1024, Loss: 0.5358
Epoch 133/1024, Loss: 0.5382
Epoch 137/1024, Loss: 0.5355
Epoch 141/102

In [37]:
dataset = "ALIBI-Chunking-512"
test_latent_dataaset(dataset)

  0%|          | 0/1024 [00:00<?, ?it/s]

Epoch 1/1024, Loss: 2.2402
Epoch 5/1024, Loss: 1.2749
Epoch 9/1024, Loss: 1.0442
Epoch 13/1024, Loss: 0.9363
Epoch 17/1024, Loss: 0.8684
Epoch 21/1024, Loss: 0.8219
Epoch 25/1024, Loss: 0.7851
Epoch 29/1024, Loss: 0.7574
Epoch 33/1024, Loss: 0.7348
Epoch 37/1024, Loss: 0.7146
Epoch 41/1024, Loss: 0.6967
Epoch 45/1024, Loss: 0.6835
Epoch 49/1024, Loss: 0.6730
Epoch 53/1024, Loss: 0.6658
Epoch 57/1024, Loss: 0.6509
Epoch 61/1024, Loss: 0.6430
Epoch 65/1024, Loss: 0.6310
Epoch 69/1024, Loss: 0.6327
Epoch 73/1024, Loss: 0.6142
Epoch 77/1024, Loss: 0.6048
Epoch 81/1024, Loss: 0.6017
Epoch 85/1024, Loss: 0.5937
Epoch 89/1024, Loss: 0.5899
Epoch 93/1024, Loss: 0.5919
Epoch 97/1024, Loss: 0.5795
Epoch 101/1024, Loss: 0.5794
Epoch 105/1024, Loss: 0.5681
Epoch 109/1024, Loss: 0.5697
Epoch 113/1024, Loss: 0.5621
Epoch 117/1024, Loss: 0.5623
Epoch 121/1024, Loss: 0.5538
Epoch 125/1024, Loss: 0.5537
Epoch 129/1024, Loss: 0.5423
Epoch 133/1024, Loss: 0.5407
Epoch 137/1024, Loss: 0.5401
Epoch 141/102

In [38]:
dataset = "ALIBI-Chunking-1024"
test_latent_dataaset(dataset)

  0%|          | 0/1024 [00:00<?, ?it/s]

Epoch 1/1024, Loss: 2.1461
Epoch 5/1024, Loss: 1.2341
Epoch 9/1024, Loss: 1.0258
Epoch 13/1024, Loss: 0.9360
Epoch 17/1024, Loss: 0.8617
Epoch 21/1024, Loss: 0.8203
Epoch 25/1024, Loss: 0.7847
Epoch 29/1024, Loss: 0.7650
Epoch 33/1024, Loss: 0.7402
Epoch 37/1024, Loss: 0.7158
Epoch 41/1024, Loss: 0.7127
Epoch 45/1024, Loss: 0.6901
Epoch 49/1024, Loss: 0.6843
Epoch 53/1024, Loss: 0.6731
Epoch 57/1024, Loss: 0.6580
Epoch 61/1024, Loss: 0.6570
Epoch 65/1024, Loss: 0.6560
Epoch 69/1024, Loss: 0.6395
Epoch 73/1024, Loss: 0.6324
Epoch 77/1024, Loss: 0.6278
Epoch 81/1024, Loss: 0.6196
Epoch 85/1024, Loss: 0.6147
Epoch 89/1024, Loss: 0.6053
Epoch 93/1024, Loss: 0.6089
Epoch 97/1024, Loss: 0.6067
Epoch 101/1024, Loss: 0.5930
Epoch 105/1024, Loss: 0.5835
Epoch 109/1024, Loss: 0.5875
Epoch 113/1024, Loss: 0.5751
Epoch 117/1024, Loss: 0.5775
Epoch 121/1024, Loss: 0.5727
Epoch 125/1024, Loss: 0.5671
Epoch 129/1024, Loss: 0.5628
Epoch 133/1024, Loss: 0.5617
Epoch 137/1024, Loss: 0.5576
Epoch 141/102

In [39]:
dataset = "ALIBI-Chunking-2048"
test_latent_dataaset(dataset)

  0%|          | 0/1024 [00:00<?, ?it/s]

Epoch 1/1024, Loss: 2.1385
Epoch 5/1024, Loss: 1.2490
Epoch 9/1024, Loss: 1.0531
Epoch 13/1024, Loss: 0.9465
Epoch 17/1024, Loss: 0.8758
Epoch 21/1024, Loss: 0.8328
Epoch 25/1024, Loss: 0.7997
Epoch 29/1024, Loss: 0.7723
Epoch 33/1024, Loss: 0.7491
Epoch 37/1024, Loss: 0.7361
Epoch 41/1024, Loss: 0.7128
Epoch 45/1024, Loss: 0.6995
Epoch 49/1024, Loss: 0.6850
Epoch 53/1024, Loss: 0.6770
Epoch 57/1024, Loss: 0.6688
Epoch 61/1024, Loss: 0.6589
Epoch 65/1024, Loss: 0.6504
Epoch 69/1024, Loss: 0.6502
Epoch 73/1024, Loss: 0.6334
Epoch 77/1024, Loss: 0.6307
Epoch 81/1024, Loss: 0.6232
Epoch 85/1024, Loss: 0.6247
Epoch 89/1024, Loss: 0.6160
Epoch 93/1024, Loss: 0.6094
Epoch 97/1024, Loss: 0.6075
Epoch 101/1024, Loss: 0.5949
Epoch 105/1024, Loss: 0.5917
Epoch 109/1024, Loss: 0.5875
Epoch 113/1024, Loss: 0.5886
Epoch 117/1024, Loss: 0.5859
Epoch 121/1024, Loss: 0.5812
Epoch 125/1024, Loss: 0.5663
Epoch 129/1024, Loss: 0.5684
Epoch 133/1024, Loss: 0.5614
Epoch 137/1024, Loss: 0.5644
Epoch 141/102